In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
import glob
import cv2
import re
import tensorflow_hub as hub

In [2]:
images=[]
for img in glob.glob("/home/sachin/ML_training/Dog_breed/resized_images/*.jpg"):
    images.append(img)
images.sort()#Returns images list in ascending order
images=images[:10000]

In [4]:
labels_df=pd.read_csv("/home/sachin/ML_training/Dog_breed/labels.csv")
labels_df.sort_values('id', axis=0, ascending=True,inplace=True)
labels_df=labels_df[:10000]
print(labels_df.shape)

(10000, 2)


In [5]:
labels_df.drop(['id'],axis=1,inplace=True)
label_no=list(range(1,121))
label_names=list(labels_df.breed.unique())

In [6]:
label_dict=dict(zip(label_names,label_no))

In [7]:
labels=tf.constant(labels_df.breed.map(label_dict))
labels=tf.one_hot(labels,120)
filename=tf.constant(images)

In [12]:
def parse_function(filename, label):
    image_string = tf.read_file(filename)

    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string, channels=3)

    resized_image = tf.image.resize_images(image, [299, 299])
    return resized_image, label

In [13]:
dataset=tf.data.Dataset.from_tensor_slices((images, labels))

In [14]:
dataset=dataset.shuffle(len(images))
dataset=dataset.map(parse_function, num_parallel_calls=4)
dataset=dataset.batch(50)
dataset=dataset.repeat(10)
init = tf.global_variables_initializer()
iterator = dataset.make_one_shot_iterator()
x,y = iterator.get_next()

In [9]:
module = hub.Module("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1")
height, width = hub.get_expected_image_size(module)


INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1'.
INFO:tensorflow:Initialize variable module/InceptionV3/Conv2d_1a_3x3/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Conv2d_1a_3x3/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_vari

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5b/Branch_2/Conv2d_0a_1x1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_5b/Branch_2/Conv2d_0a_1x1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5b/Branch_2/Conv2d_0a_1x1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_5b/Branch_2/Conv2d_0a_1x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5b/Branch_2/Conv2d_0a_1x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_5b/Branch_2/Conv2d_0a_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5b/Branch_2/Conv2d_0a_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b0

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5c/Branch_2/Conv2d_0a_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_5c/Branch_2/Conv2d_0a_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5c/Branch_2/Conv2d_0b_3x3/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_5c/Branch_2/Conv2d_0b_3x3/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5c/Branch_2/Conv2d_0b_3x3/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_5c/Branch_2/Conv2d_0b_3x3/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5c/Branch_2/Conv2d_0b_3x3/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variab

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5d/Branch_2/Conv2d_0b_3x3/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_5d/Branch_2/Conv2d_0b_3x3/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5d/Branch_2/Conv2d_0b_3x3/weights:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_5d/Branch_2/Conv2d_0b_3x3/weights
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5d/Branch_2/Conv2d_0c_3x3/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_5d/Branch_2/Conv2d_0c_3x3/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5d/Branch_2/Conv2d_0c_3x3/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/va

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6b/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_6b/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6b/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_6b/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6b/Branch_1/Conv2d_0a_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_6b/Branch_1/Conv2d_0a_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6b/Branch_1/Conv2d_0b_1x7/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff4215

...

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6e/Branch_1/Conv2d_0a_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_6e/Branch_1/Conv2d_0a_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6e/Branch_1/Conv2d_0b_1x7/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_6e/Branch_1/Conv2d_0b_1x7/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6e/Branch_1/Conv2d_0b_1x7/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_6e/Branch_1/Conv2d_0b_1x7/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6e/Branch_1/Conv2d_0b_1x7/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variab

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6e/Branch_3/Conv2d_0b_1x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_6e/Branch_3/Conv2d_0b_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6e/Branch_3/Conv2d_0b_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_6e/Branch_3/Conv2d_0b_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7a/Branch_0/Conv2d_0a_1x1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_7a/Branch_0/Conv2d_0a_1x1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7a/Branch_0/Conv2d_0a_1x1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/va

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7b/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_7b/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7b/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_7b/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7b/Branch_1/Conv2d_0a_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_7b/Branch_1/Conv2d_0a_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7b/Branch_1/Conv2d_0b_1x3/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff4215

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7b/Branch_3/Conv2d_0b_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_7b/Branch_3/Conv2d_0b_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7c/Branch_0/Conv2d_0a_1x1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_7c/Branch_0/Conv2d_0a_1x1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7c/Branch_0/Conv2d_0a_1x1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_7c/Branch_0/Conv2d_0a_1x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7c/Branch_0/Conv2d_0a_1x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variab

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7c/Branch_2/Conv2d_0d_3x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_7c/Branch_2/Conv2d_0d_3x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7c/Branch_2/Conv2d_0d_3x1/weights:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_7c/Branch_2/Conv2d_0d_3x1/weights
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7c/Branch_3/Conv2d_0b_1x1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/variables/variables' with InceptionV3/Mixed_7c/Branch_3/Conv2d_0b_1x1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7c/Branch_3/Conv2d_0b_1x1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/11d9faf945d073033780fd924b2b09ff42155763/va

In [15]:
features = module(x)

In [17]:
fc1 = tf.layers.dense(inputs=features, units=1024, activation=tf.nn.relu,name="fc1")
logits = tf.layers.dense(inputs=fc1, units=120, name="logits_layer")

In [18]:
y_prob=tf.nn.softmax(logits,name="y_prob")

In [19]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
loss=tf.reduce_mean(cross_entropy)
optimizer=tf.train.AdamOptimizer(learning_rate=0.0001)
training_op=optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [ ]:
with tf.Session() as sess:
    sess.run(init)
    epoch=0

    for batch_no in range(1,2001):
        if(batch_no%200==0):
            epoch=epoch+1      
        loss_val,accuracy_val,_=sess.run([loss,accuracy,training_op])

        print("Epoch: ",epoch,"Batch_no: ",batch_no,"Loss :===",loss_val,"     ","Accuracy:==",accuracy_val)

Epoch:  0 Batch_no:  1 Loss :=== 105.19822       Accuracy:== 0.0
Epoch:  0 Batch_no:  2 Loss :=== 57.428223       Accuracy:== 0.0
Epoch:  0 Batch_no:  3 Loss :=== 60.05724       Accuracy:== 0.0
Epoch:  0 Batch_no:  4 Loss :=== 58.15953       Accuracy:== 0.02
Epoch:  0 Batch_no:  5 Loss :=== 55.8665       Accuracy:== 0.0
Epoch:  0 Batch_no:  6 Loss :=== 46.7846       Accuracy:== 0.0
Epoch:  0 Batch_no:  7 Loss :=== 53.640625       Accuracy:== 0.0
Epoch:  0 Batch_no:  8 Loss :=== 44.563057       Accuracy:== 0.0
Epoch:  0 Batch_no:  9 Loss :=== 37.223656       Accuracy:== 0.02
Epoch:  0 Batch_no:  10 Loss :=== 39.987343       Accuracy:== 0.0
Epoch:  0 Batch_no:  11 Loss :=== 38.88841       Accuracy:== 0.02
Epoch:  0 Batch_no:  12 Loss :=== 35.807297       Accuracy:== 0.04
Epoch:  0 Batch_no:  13 Loss :=== 32.405495       Accuracy:== 0.04
Epoch:  0 Batch_no:  14 Loss :=== 35.58809       Accuracy:== 0.04
Epoch:  0 Batch_no:  15 Loss :=== 32.69845       Accuracy:== 0.0
Epoch:  0 Batch_no:  1

Epoch:  0 Batch_no:  125 Loss :=== 16.688822       Accuracy:== 0.0
Epoch:  0 Batch_no:  126 Loss :=== 15.390243       Accuracy:== 0.0
Epoch:  0 Batch_no:  127 Loss :=== 16.969257       Accuracy:== 0.02
Epoch:  0 Batch_no:  128 Loss :=== 18.261282       Accuracy:== 0.0
Epoch:  0 Batch_no:  129 Loss :=== 14.987957       Accuracy:== 0.04
Epoch:  0 Batch_no:  130 Loss :=== 14.249547       Accuracy:== 0.06
Epoch:  0 Batch_no:  131 Loss :=== 16.005       Accuracy:== 0.0
Epoch:  0 Batch_no:  132 Loss :=== 15.203221       Accuracy:== 0.02
Epoch:  0 Batch_no:  133 Loss :=== 15.3462925       Accuracy:== 0.04
Epoch:  0 Batch_no:  134 Loss :=== 15.674853       Accuracy:== 0.0
Epoch:  0 Batch_no:  135 Loss :=== 17.27391       Accuracy:== 0.04
Epoch:  0 Batch_no:  136 Loss :=== 18.212828       Accuracy:== 0.02
Epoch:  0 Batch_no:  137 Loss :=== 16.220613       Accuracy:== 0.02
Epoch:  0 Batch_no:  138 Loss :=== 16.411615       Accuracy:== 0.02
Epoch:  0 Batch_no:  139 Loss :=== 14.105072       Accur

Epoch:  1 Batch_no:  247 Loss :=== 10.85756       Accuracy:== 0.02
Epoch:  1 Batch_no:  248 Loss :=== 11.020226       Accuracy:== 0.04
Epoch:  1 Batch_no:  249 Loss :=== 12.780769       Accuracy:== 0.04
Epoch:  1 Batch_no:  250 Loss :=== 14.30811       Accuracy:== 0.04
Epoch:  1 Batch_no:  251 Loss :=== 11.9850645       Accuracy:== 0.0
Epoch:  1 Batch_no:  252 Loss :=== 11.540815       Accuracy:== 0.02
Epoch:  1 Batch_no:  253 Loss :=== 11.277153       Accuracy:== 0.1
Epoch:  1 Batch_no:  254 Loss :=== 11.136334       Accuracy:== 0.04
Epoch:  1 Batch_no:  255 Loss :=== 11.765371       Accuracy:== 0.0
Epoch:  1 Batch_no:  256 Loss :=== 11.833622       Accuracy:== 0.02
Epoch:  1 Batch_no:  257 Loss :=== 11.386421       Accuracy:== 0.02
Epoch:  1 Batch_no:  258 Loss :=== 12.944162       Accuracy:== 0.04
Epoch:  1 Batch_no:  259 Loss :=== 11.486772       Accuracy:== 0.02
Epoch:  1 Batch_no:  260 Loss :=== 10.608393       Accuracy:== 0.06
Epoch:  1 Batch_no:  261 Loss :=== 12.195339       A

Epoch:  1 Batch_no:  368 Loss :=== 11.557671       Accuracy:== 0.04
Epoch:  1 Batch_no:  369 Loss :=== 11.462298       Accuracy:== 0.02
Epoch:  1 Batch_no:  370 Loss :=== 10.360033       Accuracy:== 0.06
Epoch:  1 Batch_no:  371 Loss :=== 12.087656       Accuracy:== 0.02
Epoch:  1 Batch_no:  372 Loss :=== 12.124554       Accuracy:== 0.02
Epoch:  1 Batch_no:  373 Loss :=== 11.433978       Accuracy:== 0.02
Epoch:  1 Batch_no:  374 Loss :=== 11.308697       Accuracy:== 0.04
Epoch:  1 Batch_no:  375 Loss :=== 10.606912       Accuracy:== 0.04
Epoch:  1 Batch_no:  376 Loss :=== 12.4772415       Accuracy:== 0.02
Epoch:  1 Batch_no:  377 Loss :=== 11.653391       Accuracy:== 0.0
Epoch:  1 Batch_no:  378 Loss :=== 11.443716       Accuracy:== 0.0
Epoch:  1 Batch_no:  379 Loss :=== 10.919781       Accuracy:== 0.02
Epoch:  1 Batch_no:  380 Loss :=== 12.885301       Accuracy:== 0.02
Epoch:  1 Batch_no:  381 Loss :=== 11.563595       Accuracy:== 0.02
Epoch:  1 Batch_no:  382 Loss :=== 10.561686     

Epoch:  2 Batch_no:  490 Loss :=== 9.799721       Accuracy:== 0.06
Epoch:  2 Batch_no:  491 Loss :=== 9.664187       Accuracy:== 0.04
Epoch:  2 Batch_no:  492 Loss :=== 10.788229       Accuracy:== 0.0
Epoch:  2 Batch_no:  493 Loss :=== 9.228597       Accuracy:== 0.06
Epoch:  2 Batch_no:  494 Loss :=== 9.604758       Accuracy:== 0.04
Epoch:  2 Batch_no:  495 Loss :=== 10.644574       Accuracy:== 0.06
Epoch:  2 Batch_no:  496 Loss :=== 11.201635       Accuracy:== 0.06
Epoch:  2 Batch_no:  497 Loss :=== 10.010022       Accuracy:== 0.04
Epoch:  2 Batch_no:  498 Loss :=== 10.562426       Accuracy:== 0.02
Epoch:  2 Batch_no:  499 Loss :=== 11.068092       Accuracy:== 0.04
Epoch:  2 Batch_no:  500 Loss :=== 10.408954       Accuracy:== 0.02
Epoch:  2 Batch_no:  501 Loss :=== 9.485083       Accuracy:== 0.02
Epoch:  2 Batch_no:  502 Loss :=== 8.32958       Accuracy:== 0.08
Epoch:  2 Batch_no:  503 Loss :=== 10.199215       Accuracy:== 0.0
Epoch:  2 Batch_no:  504 Loss :=== 9.720617       Accurac